# Coco 2017 / Intake
This notebook shows how you can use [intake](https://github.com/intake/intake) to access the Coco2017 dataset.

You will need to use the patched version of [fsspec#332](https://github.com/intake/filesystem_spec/pull/332) since the ZipFiles are large.

    ```
    conda create -n intake intake s3fs intake-xarray scikit-image ipywidgets
    pip install git+https://github.com/sodre/filesystem_spec@i-zip-info-use-cache
    ```

In [1]:
import intake
from skimage import io
from ipywidgets import interact


In [2]:
coco2017 = intake.open_catalog("coco2017.yml")

In [3]:
coco2017.__getattr__("train").describe()

{'name': 'train',
 'container': 'xarray',
 'plugin': ['xarray_image'],
 'description': 'Coco 2017 Train Images',
 'direct_access': 'forbid',
 'user_parameters': [],
 'metadata': {},
 'args': {'urlpath': 'zip://**.jpg::blockcache://fast-ai-coco/train2017.zip::s3://',
  'storage_options': {'s3': {'anon': True},
   'blockcache': {'cache_storage': '{{ CATALOG_DIR }}/cache'}}}}

In [6]:
coco2017.test.discover()

{'datashape': None,
 'dtype': 'uint8',
 'shape': (40670, 480, 640, 3),
 'npartitions': 40670,
 'metadata': {'catalog_dir': '/home/pcarlos/ghq/github.com/zeroae/intake-coco2017/',
  'dims': {'channel': 3, 'concat_dim': 40670, 'x': 640, 'y': 480},
  'data_vars': {'raster': ['y', 'x', 'channel']},
  'coords': ('y', 'x', 'channel'),
  'array': 'raster'}}

In [17]:
from ipywidgets import Dropdown, IntSlider
dataset_widget = Dropdown(options=[
    (name, coco2017.__getattr__(name)) for name in coco2017
])
partition_widget = IntSlider(min=0)

def update_partition_widget(*args):
    dataset = dataset_widget.value
    partition_widget.max = dataset.discover()["shape"][0]
dataset_widget.observe(update_partition_widget, 'value')

@interact(dataset=dataset_widget, id=partition_widget)
def show(dataset, id):
    img = dataset.read_partition((id,))[0]
    io.imshow(img)

interactive(children=(Dropdown(description='dataset', options=(('unlabeled', sources:
  unlabeled:
    args:
 …